In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.densenet import DenseNet201,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexNoLegend"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = DenseNet201(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 1, 1, 1920)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 1920)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.4min


[CV] ........................... C=0.01, score=0.940313, total= 1.4min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.936255, total= 1.5min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.938370, total= 1.5min
[CV] ........................... C=0.01, score=0.924213, total= 1.5min
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.930762, total= 1.5min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.931641, total= 1.5min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.939484, total= 1.5min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  3.3min


[CV] ............................ C=1.0, score=0.908465, total= 3.3min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.930664, total= 3.3min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.933532, total= 3.3min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.939243, total= 3.3min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.945491, total= 3.4min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.929412, total= 3.4min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.923228, total= 3.4min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] ................... C=1000000000.0, score=0.916667, total= 3.2min
[CV] ................... C=1000000000.0, score=0.933464, total= 3.3min
[CV] ................... C=1000000000.0, score=0.907480, total= 3.2min
[CV] ................... C=1000000000.0, score=0.918892, total= 3.2min
[CV] ................... C=1000000000.0, score=0.927651, total= 3.1min
[CV] ................... C=1000000000.0, score=0.931102, total= 3.2min
[CV] ................... C=1000000000.0, score=0.930417, total= 3.1min
[CV] ................... C=1000000000.0, score=0.915675, total= 3.2min
[CV] ................... C=1000000000.0, score=0.933267, total= 3.1min
[CV] .................. C=10000000000.0, score=0.920898, total= 2.8min
[CV] .................. C=10000000000.0, score=0.918627, total= 2.7min
[CV] .................. C=10000000000.0, score=0.931507, total= 2.7min
[CV] .................. C=10000000000.0, score=0.907480, total= 2.7min
[CV] .................. C=10000000000.0, score=0.933071, total= 2.7min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 16.8min remaining:   40.4s


[CV] .................. C=10000000000.0, score=0.930624, total= 2.7min
[CV] .................. C=10000000000.0, score=0.919881, total= 2.7min
[CV] .................. C=10000000000.0, score=0.917659, total= 2.5min
[CV] .................. C=10000000000.0, score=0.930417, total= 2.4min
[CV] .................. C=10000000000.0, score=0.933267, total= 2.5min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 17.0min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([ 87.84760563, 203.57564249, 189.95603023, 189.99719331,
        192.16018312, 189.94374404, 193.79479144, 190.95775363,
        191.22987626, 192.85841975, 189.96586959, 190.43925724,
        159.28172395]),
 'mean_score_time': array([0.04347579, 0.04630225, 0.03755348, 0.03364758, 0.03140726,
        0.03811822, 0.03139155, 0.03118353, 0.03241313, 0.03087077,
        0.03054564, 0.02205601, 0.01446049]),
 'mean_test_score': array([0.93537885, 0.93458958, 0.92649961, 0.92324388, 0.9234412 ,
        0.92403315, 0.92294791, 0.92334254, 0.92511839, 0.92284925,
        0.92353986, 0.92353986, 0.92432912]),
 'mean_train_score': array([0.98054249, 0.99768701, 0.99975885, 0.99974792, 0.99974789,
        0.99974791, 0.99971503, 0.99973696, 0.99973696, 0.99974792,
        0.99978078, 0.99976981, 0.99973694]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.93538, std: 0.00651, params: {'C': 0.01},
 mean: 0.93459, std: 0.00647, params: {'C': 0.1},
 mean: 0.92650, std: 0.00858, params: {'C': 1.0},
 mean: 0.92324, std: 0.00946, params: {'C': 10.0},
 mean: 0.92344, std: 0.00834, params: {'C': 100.0},
 mean: 0.92403, std: 0.00870, params: {'C': 1000.0},
 mean: 0.92295, std: 0.00888, params: {'C': 10000.0},
 mean: 0.92334, std: 0.01007, params: {'C': 100000.0},
 mean: 0.92512, std: 0.00910, params: {'C': 1000000.0},
 mean: 0.92285, std: 0.00938, params: {'C': 10000000.0},
 mean: 0.92354, std: 0.00826, params: {'C': 100000000.0},
 mean: 0.92354, std: 0.00843, params: {'C': 1000000000.0},
 mean: 0.92433, std: 0.00823, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.9354
